### Motivation
* This notebook is to create a trusted California State block groups on a feature layer
We will try to see if we can use the existing feature layer created from Esri :  
Feature Layer: https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer  
Feature Layer Details: https://esri.maps.arcgis.com/home/item.html?id=2f5e592494d243b0aa5c253e75e792a4
* Using this layer, we would pick CA County FIPS data  
* Enrich the Dataframe with consumer spending and other variables

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geoenrichment import enrich, Country
from feature_layer_utils import get_enrichment_variables
import itertools
import sys
sys.path.append('../../')
from utils import get_config

In [21]:
app_id = get_config("arcgis","clientid")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=YYdyhiSn83R7vhPbN5NpP4g3JWWlYT&allow_verification=false
Enter code obtained on signing in using SAML: ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


#### Selecting only San Diego and Imperial County block groups from US block groups

In [4]:
fls = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer"

In [5]:
flc = FeatureLayerCollection(fls)

In [6]:
fl_url = flc.layers[0].url

In [7]:
us_block_grp_fl = FeatureLayer(fl_url)

Selecting FIPS=`06` which is for California State

In [8]:
ca_fip = '06'

In [10]:
ca_block_grp_fs = us_block_grp_fl.query(
    where=f"STATE_FIPS='{ca_fip}'"
)

In [11]:
ca_block_grp_fs_sdf = ca_block_grp_fs.sdf
print(f"Shape: {ca_block_grp_fs_sdf.shape}")
ca_block_grp_fs_sdf.head(4)

Shape: (25587, 14)


,OBJECTID,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,BLOCKGROUP_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Area,Shape__Length,SHAPE
0,11394,CA,06,001,06001,400100,1,060014001001,1789,805.9,2.22,0.000588,0.11378,"{""rings"": [[[-122.24244320135, 37.882207966106..."
1,11395,CA,06,001,06001,400200,1,060014002001,1022,9290.9,0.11,0.000029,0.031773,"{""rings"": [[[-122.241923197175, 37.85181896382..."
2,11396,CA,06,001,06001,400200,2,060014002002,979,8158.3,0.12,0.000031,0.03383,"{""rings"": [[[-122.256879198933, 37.84293996164..."
3,11397,CA,06,001,06001,400300,1,060014003001,1123,11230.0,0.1,0.000027,0.025534,"{""rings"": [[[-122.251354197915, 37.84015996211..."


For enrichment, we can provide the feature set or the spatially enabled df.
Let us just pass down the sdf since we have visualized it already.

In [12]:
ca_block_groups_fl = ca_block_grp_fs_sdf.spatial.to_featurelayer(title='California State Block Groups',
                                                                 gis=gis,
                                                                 folder='nourish_gis',
                                                                 tags=['California','BlockGroups','County'],
                                                                 service_name='California State Block Groups'
                                                                )
ca_block_groups_fl


<Item title:"California State Block Groups" type:Feature Layer Collection owner:akale_UCSDOnline>